Michael O'Donnell - 02/08/2024

STC 510 - Module 5 Essentials 

Logisitc Regression 

For the essentials assignment, this project builds off of the basics of the basics of the assignment. I import and clean the data the same. I borrowed that code from myself for the sake of simplicity. I used this stack overflow question: 

https://stackoverflow.com/questions/12693438/alternatives-to-naive-bayes-algorithm

to get ideas for 2 other algorithms/methods to classify text in python. In this page, one author suggests to use Logistic Regression and Support Vector Machine. This portion of the project follows this article:

https://realpython.com/python-keras-text-classification/

on Logistic Regression. Me using this builds off my code from the other assignment under a line break. 

In [77]:
#import statements
import json
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import nltk
from nltk import word_tokenize, sent_tokenize, pos_tag
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.linear_model import LogisticRegression

In [78]:
#setting the stop words
english_stopwords = set(stopwords.words('english') + list(punctuation) + ['..','...','nbsp','n\'t'])

In [79]:
#iniatalizing the lemmatizer
lemmatizer = WordNetLemmatizer()

In [80]:
#bringing in the jeopardy JSON file
with open('jeopardy.json') as jeopardy_file:
    jeopardy_contents = jeopardy_file.read()

In [81]:
#parsing the jeopardy JSON file so it can read by python and put into a dataframe
parsed_jeopardy = json.loads(jeopardy_contents)

In [82]:
#puts the JSON questions into a dataframe
df = pd.DataFrame(parsed_jeopardy)

In [83]:
#Cleans the value column and sets the values to integers
df['value'] = df['value'].str.replace('$','')
df['value'] = df['value'].str.replace(',','')
df['value'] = df['value'].str.strip()
df['value'] = pd.to_numeric(df['value'])

In [84]:
#Adds a new column that uses the explained above logic that gives easy questions a 1 and hard questions a 0
df['difficulty'] = np.where(((df['round']=='Jeopardy!') & (df['value'] <= 800)) | ((df['round']=='Double Jeopardy!') & (df['value'] <= 1200)),1,0)

In [85]:
#This is a function that takes each cell given to it from a dataframe that then goes and makes it lowercase,
# tokenizes it, lemmantizes it and puts it back into the original string of the cell 
def manipulate_text(text):
    text = text.lower()
    toke_text = word_tokenize(text)
    wordlist = []
    for word in toke_text:
        if word not in english_stopwords:
            wordlist.append(word)
    wordlist2 = []
    for eachword in wordlist:
        wordlist2.append(lemmatizer.lemmatize(eachword))
    text = [' '.join(wordlist2)]
    text = str(text)
    return text

In [86]:
#This puts the column of question that I want to manipulate for training in the above function
df['question'] = df['question'].apply(manipulate_text)

----------------------------------------

This is where I start the processing for Logistic Regression following along with the above article that was mentioned

In [87]:
#Gets numpy arrays for the model from the dataframe which houses my data which is easier to work with in this context
question = df['question'].values
difficulty = df['difficulty'].values

In [88]:
#Splits the data into training and test data sets. This has a 75% training size and 25% test size split
#I found that the 75% training size yielded better results later on. I tried 50/50 and 70/30 splits.
question_train, question_test, y_train, y_test = train_test_split(question, y, test_size=.25, random_state=1000)

In [89]:
#Imports the CountVectorizer to vectorize the bag of words
from sklearn.feature_extraction.text import CountVectorizer

In [90]:
#Intializes the vectorizer
vectorizer = CountVectorizer()

In [91]:
#Vecotrizes our training questions
vectorizer.fit(question_train)

CountVectorizer()

In [92]:
#Transforms the vectorizations
X_train = vectorizer.transform(question_train)
X_test = vectorizer.transform(question_test)

In [93]:
#Imports the Logistic Regression module
from sklearn.linear_model import LogisticRegression

In [94]:
#Intiaties the module with a max iteration of 1000. I am not sure what this does exactly but it needed it to fully
#run. This is a solution chatGPT told me to because I got a warning message originally. 
classifier = LogisticRegression(max_iter=1000)

In [95]:
#Trains the vectorized questions against the 1s and 0s of the difficulty data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [96]:
#Sets the score of the text classification
score = classifier.score(X_test, y_test)

In [97]:
#prints the score
print(score)

0.7820699574060074


The score was very similar to the Naive Bayes classification, so I was thrilled to see this result. I believe a second high score from a different sorting algorithm helps show how resilient my sorting classifications were for the text originally. 